In [1]:
import numpy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
trajectories = pd.read_csv("TrajDataV2_20120728.txt",  delimiter="\t", names=('Date', 'wind', 'time', 'speed', 'altitude','latitude', 'longitude', 'nan'))
trajectories.head()

,Date,wind,time,speed,altitude,latitude,longitude,nan
845.0,28.0,0.0,1.0,470.31621,39000.0,40.692500,-74.168667,NaN
845.0,28.0,0.0,2.0,470.31621,39000.0,40.784686,-74.019193,NaN
845.0,28.0,0.0,3.0,470.31621,39000.0,40.877281,-73.868705,NaN
845.0,28.0,0.0,4.0,470.31621,39000.0,40.969822,-73.717979,NaN
845.0,28.0,0.0,5.0,470.31621,39000.0,41.062186,-73.566848,NaN
